<a href="https://colab.research.google.com/github/Hyun-Jun-Lee/Review_Sentiment_Analysis/blob/main/Movie_Review_Seniment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
!pip install konlpy

In [3]:
import pandas as pd
import pymongo
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split


# Read Data

In [4]:
df = pd.read_csv('/content/drive/MyDrive/crawling_review.csv')
stopwords = pd.read_csv('/content/drive/MyDrive/stopword.csv')
stopwords = list(stopwords['stopdword'].values)

In [5]:
df.sample(10)

,Unnamed: 0,star,comment,label
322,338,9,앤드류 가필드 연기가 정말 좋다 조나단 라슨에 대해 하나도 모르고 봤는데 재밌었다 ...,1
1477,1593,10,음스러운 장면도 있었지만 재밌게 잘 봤다,1
871,943,10,이 영화 제 인생 최고의 영화입니다,1
8548,9051,6,이것도 미화 아닌가,0
376,395,10,대박 꿀잼이에요 저희 학교 단체로 봤는데 애들 다 만족했어요 ㅠㅠㅠ 다들보새요,1
8924,9435,4,지루하고 단조로우면서 재미없네요 마무리마저 미국식 신파라니,0
1666,1795,10,명작인데 평점점 뭐냐 ㅋ,1
3143,3378,7,재밌게 봤어요 편도 보고 싶어지네요,1
32,37,10,다양한 로맨스를 무겁지 않고 유쾌하게 그린 영화에요 역할마다 특색있고 재미있습니다,1
547,581,8,마지막 억지 신파만 아니었으면 점 주고 싶은 영화 차라리 주요 캐릭터들 살려서 속편...,1


In [5]:
stopwords[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

In [6]:
# NA값 제거
df = df.dropna(axis=0)
df.isna().sum()

Unnamed: 0    0
star          0
comment       0
label         0
dtype: int64

In [7]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
df = df.sample(frac=1).reset_index()
df.tail()

,index,star,comment,label
9457,9274,2,연기자 분들에 연기는 최고였지만 감독이 어떤걸 표현하는지 내용이 이해가 안가네요,0
9458,9230,6,소모성 캐릭터조차도 쉽게 내뱉는 철학적인 대사와 어디서 많이신세계 친구봐왔던 분위기...,0
9459,604,8,오 역시 믿고보는 제작진인가 간만에 쏘 만족스러운 스케일이였네요,1
9460,2830,8,작품이다왜 제목이 크레이지 샤크냐,1
9461,6115,8,영화관에서 본걸 다행으로 여겨요 총소리 사운드와 배우들의 연기가 돋보였습니다,1


In [9]:
df.drop('index', axis=1, inplace=True)

In [10]:
df

,star,comment,label
0,2,남감독의 레즈비언 환상으로 이뤄진 내용뿐,0
1,4,윤계상 연기는 훌륭했다 하지만 스토리 결말이 아쉽다,0
2,10,다시봐도 너무 재미있고 잘봤습니다다섯군대의 전투가 기다려지는군요,1
3,10,마음이 너무 아파요 ㅜㅜ,1
4,6,시대에 맞는 영화네요,0
...,...,...,...
9457,2,연기자 분들에 연기는 최고였지만 감독이 어떤걸 표현하는지 내용이 이해가 안가네요,0
9458,6,소모성 캐릭터조차도 쉽게 내뱉는 철학적인 대사와 어디서 많이신세계 친구봐왔던 분위기...,0
9459,8,오 역시 믿고보는 제작진인가 간만에 쏘 만족스러운 스케일이였네요,1
9460,8,작품이다왜 제목이 크레이지 샤크냐,1


In [11]:
# train_test_split
target= 'label'
features = df.drop(columns= target).columns

train, test = train_test_split(df, test_size=0.2, random_state=2021, stratify=df[target])

In [12]:
print(len(train), len(test))

7569 1893


# Preprocessing

## 토큰화 및 stopword 제거

In [ ]:
!pip install jpype1==0.7.0

In [13]:
# 한국 형태소 분석기
okt = Okt()

In [14]:
X_train = []
for sentence in tqdm(train['comment']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 7569/7569 [00:44<00:00, 169.50it/s]


In [15]:
X_test = []
for sentence in tqdm(test['comment']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 1893/1893 [00:09<00:00, 191.53it/s]


## 정수 인코딩

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [17]:
# 높은 정수가 부여된 단어가 등장 빈도수가 낮음을 의미
print(tokenizer.word_index)

{'하다': 1, '영화': 2, '보다': 3, '도': 4, '좋다': 5, '는': 6, '은': 7, '너무': 8, '이다': 9, '없다': 10, '연기': 11, '재밌다': 12, '한': 13, '적': 14, '되다': 15, '배우': 16, '스토리': 17, '만': 18, '다': 19, '않다': 20, '진짜': 21, '아니다': 22, '생각': 23, '나오다': 24, '자다': 25, '인': 26, '수': 27, '안': 28, '게': 29, '정말': 30, '만들다': 31, '연출': 32, '더': 33, '사람': 34, '볼': 35, '하고': 36, '가다': 37, '보고': 38, '감동': 39, '못': 40, '그냥': 41, '고': 42, '요': 43, '감독': 44, '님': 45, '최고': 46, '들다': 47, '재미있다': 48, '액션': 49, '느낌': 50, '내': 51, '싶다': 52, '잘': 53, '말': 54, '모르다': 55, '아쉽다': 56, '이렇다': 57, '내용': 58, '거': 59, '점': 60, '마지막': 61, '평점': 62, '그렇다': 63, '작품': 64, '오다': 65, '많다': 66, '중': 67, '이야기': 68, '넘다': 69, '재미': 70, '다시': 71, 'ㅠㅠ': 72, '정도': 73, '서': 74, '지루하다': 75, '지': 76, '장면': 77, '성': 78, '많이': 79, '주인공': 80, '없이': 81, '사랑': 82, '몰입': 83, '뭐': 84, '매력': 85, '기대': 86, '처음': 87, '영상': 88, 'ㅋㅋ': 89, '알다': 90, '상미': 91, '주다': 92, '느끼다': 93, '나다': 94, '현실': 95, '분': 96, '꼭': 97, '끝': 98, '아깝다': 99, '임': 100, '인데': 101, '역시': 102

In [18]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 10538
등장 빈도가 2번 이하인 희귀 단어의 수: 6597
단어 집합에서 희귀 단어의 비율: 62.6020117669387
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 7.416582727610958


In [19]:
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt+1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 3942


In [20]:
# 빈도수 1인 단어는 제거
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [21]:
y_train = np.array(train['label'])
y_test = np.array(test['label'])

In [22]:
print(len(X_train))
print(len(y_train))

7569
7569


In [23]:
print(X_train[:3])

[[16, 61, 847, 240, 650, 253, 110, 64, 3112, 1, 279, 90, 189, 22, 124, 403, 2], [3113, 100], [84, 43, 280, 22, 444]]


In [24]:
# 낮은 빈도수로만 구성되있었던 row 제거
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(X_test) if len(sentence) < 1]

X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
X_test = np.delete(X_test, drop_test, axis=0)
y_test = np.delete(y_test, drop_test, axis=0)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [25]:
print(len(X_train))
print(len(y_train))

7534
7534


In [26]:
print(len(X_test))
print(len(y_test))

1883
1883


## 패딩

In [27]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

리뷰의 최대 길이 : 244
리뷰의 평균 길이 : 13.542075922484736


In [30]:
# 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇%인지 확인하는 함수
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [35]:
max_len=40
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 40 이하인 샘플의 비율: 95.619856649854


In [36]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

# Modeling

In [37]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [38]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

In [39]:
#  검증 데이터 손실이 4회 증가하면 학습을 조기 종료(val_loss 증가는 과적합 징후)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

# ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [41]:
# validation_split : 훈련 데이터 중 20%를 검증 데이터로 사용하며 정확도 확인
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=30, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/30
94/95 [============================>.] - ETA: 0s - loss: 0.2050 - acc: 0.9209
Epoch 00001: val_acc did not improve from 0.81088
95/95 [==============================] - 14s 112ms/step - loss: 0.2048 - acc: 0.9210 - val_loss: 0.5317 - val_acc: 0.7910
Epoch 2/30
94/95 [============================>.] - ETA: 0s - loss: 0.1819 - acc: 0.9299
Epoch 00002: val_acc did not improve from 0.81088
95/95 [==============================] - 10s 102ms/step - loss: 0.1818 - acc: 0.9300 - val_loss: 0.5755 - val_acc: 0.7950
Epoch 3/30
94/95 [============================>.] - ETA: 0s - loss: 0.1653 - acc: 0.9357
Epoch 00003: val_acc did not improve from 0.81088
95/95 [==============================] - 10s 103ms/step - loss: 0.1653 - acc: 0.9356 - val_loss: 0.5700 - val_acc: 0.7890
Epoch 4/30
94/95 [============================>.] - ETA: 0s - loss: 0.1521 - acc: 0.9427
Epoch 00004: val_acc did not improve from 0.81088
95/95 [==============================] - 10s 103ms/step - loss: 0.1522 - acc: 

In [42]:
best_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (best_model.evaluate(X_test, y_test)[1]))

59/59 [==============================] - 2s 17ms/step - loss: 0.4698 - acc: 0.8163

 테스트 정확도: 0.8163
